# Pre-Processing Labor Force Survey Data

The [OpenDP documentation](https://docs.opendp.org) uses data derived from the [EU Labor Force Survey](https://ec.europa.eu/eurostat/web/microdata/public-microdata/labour-force-survey):

> Public microdata, also referred to as public use files, for the EU Labour force survey (LFS) were created to enable interested parties to become familiar with microdata.
>
> At the same time, the privacy of respondents had to be protected. The structure of public microdata is the same as that of research microdata available in scientific use files.
>
> Public microdata enable researchers and trainers to develop programmes using the same formats and variable names as for the actual LFS scientific use files. The files have been designed so that programmes and procedures created with public microdata will also work with scientific use files.

Code developed to work with a public microdata set like this could also be used with the scientific use files, and we believe that differential privacy would be a good tool to ensure that statistics derived from scientific use files could not inadvertently reveal personal information.

To reduce the download size for the tutorial, we've preprocessed the data by selecting a single country (France), dropping unused columns, sampling a subset of the rows, and concatenating the result into a single CSV. The code we'll present in the tutorials could be run on the original public microdata, or for that matter, the full private scientific use files.

In [ ]:
![ -e FR_PUF_LFS.zip ] || wget https://ec.europa.eu/eurostat/cache/website/microdata/public-microdata-lfs/FR_PUF_LFS.zip
!unzip -q FR_PUF_LFS.zip -d FR_PUF_LFS

In [1]:
import pandas as pd 
from pathlib import Path
import json

projection = json.load(open("projection.json"))

dfs = []
for csv_path in Path('FR_PUF_LFS').glob('*Q*.csv'):
    dfs.append(pd.read_csv(csv_path))

df = pd.concat(dfs, ignore_index=True)

# Drop empty rows:
df.dropna(axis=0, how='all', inplace=True)
df = df[df['REFYEAR'] > 2004] # all notebooks are based on 9 years of data
df = df[df['REFYEAR'] < 2014] # all notebooks are based on 9 years of data
# We'll subset columns later

df

,COEFF,QUARTER,REFYEAR,REFWEEK,INTWEEK,COUNTRY,REGION,DEGURBA,HHINST,INTWAVE,...,LEAVCLAS,NACE1D,NACE2J1D,NACEPR1D,AGERESID,IS881D,IS88PR3D,IS88PR1D,IS883D,HHTYPE
87773,0.978310,Q3,2006,30,32.0,FR,NaN,0.0,9.0,5,...,NaN,9.0,9.0,9.0,NaN,NaN,NaN,NaN,NaN,1
87774,0.704042,Q3,2006,38,39.0,FR,NaN,0.0,9.0,2,...,NaN,NaN,9.0,9.0,NaN,NaN,NaN,NaN,NaN,1
87775,0.146362,Q3,2006,32,33.0,FR,NaN,0.0,9.0,1,...,8.0,9.0,9.0,9.0,NaN,NaN,NaN,NaN,NaN,1
87776,0.255195,Q3,2006,34,35.0,FR,NaN,3.0,9.0,5,...,NaN,9.0,9.0,9.0,NaN,NaN,NaN,NaN,NaN,1
87777,0.765000,Q3,2006,33,34.0,FR,NaN,0.0,9.0,2,...,NaN,9.0,9.0,9.0,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4159827,0.654399,Q1,2008,6,8.0,FR,NaN,0.0,9.0,1,...,NaN,F,9,9,NaN,NaN,NaN,NaN,NaN,1
4159828,1.392808,Q1,2008,2,3.0,FR,NaN,0.0,9.0,5,...,NaN,H,9,9,NaN,NaN,NaN,NaN,NaN,1
4159829,0.355306,Q1,2008,1,2.0,FR,NaN,0.0,9.0,4,...,NaN,G,9,9,NaN,NaN,NaN,NaN,NaN,1
4159830,0.854188,Q1,2008,10,11.0,FR,NaN,0.0,9.0,5,...,NaN,H,9,9,NaN,NaN,NaN,NaN,NaN,1


In [2]:
# Number of years in the data (all the docs are built on 9 years, so lets try to keep it that way):
assert df["REFYEAR"].nunique() == 9

In [3]:
# Convert quarter to integer:
df['QUARTER'] = df['QUARTER'].apply(lambda quarter: int(quarter.replace('Q', '')))
# construct a birth year:
df['BIRTHYEAR'] = df['REFYEAR'] - df['AGE']

In [4]:
import numpy as np
np.random.seed(1)
# num unique random numbers = total number of rows / product of cardinalities of pseudo identifiers
# increase to avoid collisions, which concentrates contributions closer to zero
df['RANDOM'] = np.random.randint(1, 14678, df.shape[0])
# hash pseudo identifiers with random numbers so that generated ids will have consistent pseudo identifiers
# modulo a larger number to shorten identifiers (and introducing some collisions)
df['PIDENT'] = df[[*projection["pseudo_identifiers"], 'RANDOM']].apply(lambda x: hash(tuple(x)), axis = 1).rank(method='dense').astype(int)
df.sort_values(by=['PIDENT'], inplace=True)

In [5]:
# check distribution of identifiers
df['PIDENT'].value_counts().value_counts()

count
1     680705
2     412318
3     282625
4     166072
5      82068
6      36524
7      14323
8       5286
9       1710
10       480
11       149
12        35
13         9
15         1
14         1
Name: count, dtype: int64

Many pseudo-individuals contribute one record, fewer people get surveyed many times.

In [6]:
df_small = df[projection["keep"]]
# sampling to reduce size not necessary because relatively few columns are kept 
# df_small = df_small.sample(50_000, random_state=1)
df_small.to_csv('V2_FR_LFS.csv', index=False)

In [7]:
!zip V2_FR_LFS.csv.zip V2_FR_LFS.csv

updating: V2_FR_LFS.csv (deflated 86%)
